In [2]:
import pickle
# path = "data/Physics/0_uncopynode.pkl"
# with open(path, 'rb') as f:
#     data = pickle.load(f)
# print(data.edge_index.max())
# print(len(data.x))

In [41]:
# print(data.edge_index.shape, data.train_mask.shape)
# s = set()
# for e in data.edge_index[1]:
#     s.add(e.item())
# print(len(s))
# data.edge_index
# data.train_mask

# path = 'data/SBM/0_uncopynode.pkl'
# with open(path, 'rb') as f:
#     data = pickle.load(f)
# print(data.y.min(),data.y.max())
# print(data.edge_index[0].max())
# print(data.edge_index[1].max())
# print(len(data.x[0]))
import torch
x = torch.Tensor([0,0])
print(x.to('cuda').grad)


None


In [3]:
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

path_sbm_train = "data/SBM/SBM_CLUSTER_train.pkl"
path_sbm_val = "data/SBM/SBM_CLUSTER_val.pkl"
path_sbm_test = "data/SBM/SBM_CLUSTER_test.pkl"
with open(path_sbm_train, 'rb') as f:
    sbm_train = pickle.load(f)
with open(path_sbm_val, 'rb') as f:
    sbm_val = pickle.load(f)
with open(path_sbm_test, 'rb') as f:
    sbm_test = pickle.load(f)
# sbm_train

In [4]:
# for tmp in sbm_train:
#     print(len(tmp.node_feat))
# sbm_train[0].node_feat
# edge_index = torch.tensor([[],[]])
# print(edge_index)
# print(torch.cat((edge_index, torch.nonzero(sbm_train[0].W).T+100), 1))
# x = torch.BoolTensor([0]*10)
# x[1:3] = True
# print(x)
# print([1,2]*2)
s_train = 0
for tmp in sbm_train:
    s_train += len(tmp.node_feat)
s_val = 0
for tmp in sbm_val:
    s_val += len(tmp.node_feat)
s_test = 0
for tmp in sbm_test:
    s_test += len(tmp.node_feat)
print(s_train, s_val, s_test, s_train + s_val+s_test)

1168087 116896 116820 1401803


In [1]:
from torch_geometric.data import Data
K=1000
NBTRAIN = len(sbm_train)
NBVAL = len(sbm_val)
NBTEST = len(sbm_test)

import torch
for k in range(K):
    print(k)
    idx_base=0
    edge_index = torch.LongTensor([[],[]])
    x = torch.tensor([])
    y = torch.LongTensor([])
    for idx_graph in range(k*int(NBTRAIN/K),(k+1)*int(NBTRAIN/K)):
        sub_graph = sbm_train[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    # print(x.size(), edge_index, idx_base)
    idx_train = idx_base
    for idx_graph in range(k*int(NBVAL/K), (k+1)*int(NBVAL/K)):
        sub_graph = sbm_val[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    # print(x.size(), edge_index, idx_base)
    idx_val = idx_base
    for idx_graph in range(k*int(NBTEST/K), (k+1)*int(NBTEST/K)):
        sub_graph = sbm_test[idx_graph]
        edge_index = torch.cat(
            (edge_index, torch.nonzero(sub_graph.W).T+idx_base), 1)
        x = torch.cat((x, torch.unsqueeze(sub_graph.node_feat, 1)), 0)
        y = torch.cat((y, sub_graph.node_label), 0)
        idx_base += len(sub_graph.node_feat)
    # print(x.size(), edge_index, idx_base)
    idx_test = idx_base

    x = torch.zeros(x.shape[0], 7).scatter(1, torch.tensor(x,dtype=torch.int64), 1)[:, 1:]
    train_mask = torch.BoolTensor([0]*idx_base)
    train_mask[0:idx_train] = True
    val_mask = torch.BoolTensor([0]*idx_base)
    val_mask[idx_train:idx_val] = True
    test_mask = torch.BoolTensor([0]*idx_base)
    test_mask[idx_val:idx_test] = True
    with open("data/SBM/{}_uncopynode.pkl".format(k), "wb") as f:
        pickle.dump(Data(edge_index=edge_index, x = x, y = y, train_mask = train_mask, val_mask = val_mask, test_mask = val_mask), f)
    # break


In [28]:
# from models import Gat
# model = Gat(6,5)
path = 'data/SBM/0_uncopynode.pkl'
with open(path, 'rb') as f:
    data = pickle.load(f).to('cuda:0')
input("ending?")
# print(data.x.size(),data.edge_index)
# output = model(data.x,data.edge_index)
# output

torch.Size([1545, 6]) tensor([[   0,    0,    0,  ..., 1544, 1544, 1544],
        [   1,    8,    9,  ..., 1535, 1536, 1537]])


tensor([[-0.0098,  0.0017,  0.0037, -0.0007, -0.0068],
        [-0.0114,  0.0024,  0.0004, -0.0016, -0.0068],
        [-0.0093,  0.0013,  0.0036, -0.0011, -0.0052],
        ...,
        [-0.0179,  0.0027,  0.0038, -0.0019, -0.0082],
        [-0.0157,  0.0015,  0.0032, -0.0009, -0.0094],
        [-0.0168,  0.0040,  0.0111, -0.0007, -0.0146]],
       grad_fn=<AsStridedBackward0>)